In [ ]:
import os
import shutil

if not os.path.isdir("./libs"):
    os.mkdir("./libs")

In [ ]:
%%writefile ./libs/util_bd.py
import mysql.connector
import myloginpath
import pandas as pd

def resultados_query(query):
    conf = myloginpath.parse('tesis2')
    conn = mysql.connector.connect(**conf, db="tesis2")
    cursor = conn.cursor()
    cursor.execute(query)
    resultado = cursor.fetchall()
    conn.close()
    return resultado

def ejecutar_query(query):
    conf = myloginpath.parse('tesis2')
    conn = mysql.connector.connect(**conf, db="tesis2")
    cursor = conn.cursor()
    cursor.execute(query)
    conn.close()

def mostrar_resultado_query(query):
    conf = myloginpath.parse('tesis2')
    conn = mysql.connector.connect(**conf, db="tesis2")
    df = pd.read_sql_query(query, conn)
    display(df)
    conn.close()

In [2]:
%%writefile ./libs/util_fasta.py
import os

def generar_fasta(secuencias, archivo, tamanio_por_linea=80):
    t_tamanio = tamanio_por_linea
    f = open(archivo ,"w+")
    for transcrito in secuencias:
        f.write(">%s\n" % (transcrito[0].strip().upper()))
        seq = transcrito[1]
        t_partes = [seq[i:i+t_tamanio] for i in range(0, len(seq), t_tamanio)]
        for t_parte in t_partes:
            f.write("%s\n" % (t_parte))
    f.close()

def leer_fasta(archivo, limite = 0):
    transcritos = {}
    cod_secuencia = ""
    secuencia = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if secuencia != "":
                transcritos[cod_secuencia] = secuencia
                secuencia = ""
                limite -= 1
                if limite == 0:
                    break
            cod_secuencia = linea.rstrip("\n").lstrip(">").strip().upper()
        else:
            secuencia += linea.rstrip("\n")
    if secuencia != "":
        transcritos[cod_secuencia] = secuencia
        secuencia = ""
    f.close()
    return transcritos

def leer_fasta_list(archivo, limite = 0):
    transcritos = list()
    cod_secuencia = ""
    secuencia = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if secuencia != "":
                transcritos.append((cod_secuencia, secuencia))
                secuencia = ""
                limite -= 1
                if limite == 0:
                    break
            cod_secuencia = linea.rstrip("\n").lstrip(">").strip().upper()
        else:
            secuencia += linea.rstrip("\n")
    if secuencia != "":
        transcritos.append((cod_secuencia, secuencia))
        secuencia = ""
    f.close()
    return transcritos

Overwriting ./libs/util_fasta.py


In [4]:
%%writefile ./libs/util_caracteristicas.py
import os
import util_fasta
from Bio.SeqUtils import GC
import csv
from sklearn.externals.joblib import dump, load
#import random
#import string
#import shutil

def generar_modelo_CPAT(archivo_lncRNA, archivo_PCT, archivo_CDS, carpeta_cpat):
    _generar_modelo_CPAT_hexamer(archivo_lncRNA, archivo_CDS, carpeta_cpat)
    _generar_modelo_CPAT_logit(archivo_lncRNA, archivo_PCT, carpeta_cpat)

def _generar_modelo_CPAT_hexamer(archivo_lncRNA, archivo_CDS, carpeta_cpat):
    script = "~/anaconda3/bin/make_hexamer_tab.py"
    fasta_cds = "'" + archivo_CDS + "'" 
    fasta_lncRNA = "'" + archivo_lncRNA + "'"
    salida = "'" + carpeta_cpat + "/hexamer.tsv" + "'"
    comando = "{} -c {} -n {} > {}".format(script, fasta_cds, fasta_lncRNA, salida)
    os.system(comando)
    
def _generar_modelo_CPAT_logit(archivo_lncRNA, archivo_PCT, carpeta_cpat):
    script = "~/anaconda3/bin/make_logitModel.py"
    hexamer = "'" + carpeta_cpat + "/hexamer.tsv" + "'"
    fasta_pct = "'" + archivo_PCT + "'" 
    fasta_lncRNA = "'" + archivo_lncRNA + "'"
    salida = "'" + carpeta_cpat + "/fold" + "'"
    comando = "{} -x {} -c {} -n {} -o {}".format(script, hexamer, fasta_pct, fasta_lncRNA, salida)
    os.system(comando)
    
def ejecutar_diamond(archivo_entrada, diamond_db, archivo_salida):
    script = "~/anaconda3/bin/diamond"
    diamond_bd = "'" + diamond_db + "'"
    salida = "'" + archivo_salida + "'"
    comando = "{} blastx -d {} -q {} -o {} -k 5 --gapopen 11 --gapextend 1 --more-sensitive -f 6 qseqid pident length qframe qstart qend sstart send evalue bitscore".format(script, diamond_bd, archivo_entrada, salida)
    os.system(comando)

def ejecutar_cpat(archivo_entrada, carpeta_cpat, archivo_salida):
    script = "~/anaconda3/bin/cpat.py"
    logit = "'" + carpeta_cpat + "/fold.logit.RData" + "'"
    hexamer = "'" + carpeta_cpat + "/hexamer.tsv" + "'"
    salida = "'" + archivo_salida + "'"
    comando = "{} -g {} -d {} -x {} -o {}".format(script, archivo_entrada, logit, hexamer, salida)
    os.system(comando)
    
def generar_features_base(archivo_entrada, archivo_cpat, archivo_diamond, archivo_salida):
    transcritos = util_fasta.leer_fasta(archivo_entrada)
    transcript_dict = {}
    for k in transcritos.keys():
        transcript_dict[k.strip().upper()] = {
            "length" : len(transcritos[k]),
            "gc" : GC(transcritos[k]),
            "orf_length" : 0,
            "orf_coverage" : float(0),
            "hexamer_score" : float(0),
            "fickett_score" : float(0),
            "identity" : float(0),
            "align_length" : float(0),
            "align_perc_len" : float(0),
            "align_perc_orf" : float(0)
        }
    
    #adaptado de https://github.com/gbgolding/crema/blob/master/bin/featuresetup_module.py
    with open(archivo_cpat, "r") as f:
        cpat_reader = csv.reader(f, delimiter=("\t"))
        next(cpat_reader, None) # skip header
        for row in cpat_reader:
            cod_secuencia = row[0]
            transcript_dict[cod_secuencia]["orf_length"] = float(row[2])
            transcript_dict[cod_secuencia]["orf_coverage"] = float(row[2])/float(transcript_dict[cod_secuencia]["length"])
            transcript_dict[cod_secuencia]["fickett_score"] = float(row[3])
            transcript_dict[cod_secuencia]["hexamer_score"] = float(row[4])
    
    with open(archivo_diamond, "r") as f:
        tab_reader = csv.reader(f, delimiter=("\t"))
        line_1 = next(tab_reader)
        first = line_1[0].upper()
        score = [float(line_1[9])]
        with_len = [[first, float(line_1[1]), float(line_1[2]), float(line_1[3]), float(line_1[9])]] # name identity length frame score
        for row in tab_reader:
            if row[0].upper() == first:
                score.append(float(row[9]))
                with_len.append([row[0].upper(), float(row[1]), float(row[2]), float(row[3]), float(row[9])])
            else:
                transcript_dict[first]["identity"] = float(0)
                transcript_dict[first]["align_length"] = float(0)
                max_value = max(score)
                max_index = score.index(max_value)
                max_len_ident = with_len[max_index]
                if max_len_ident[3] > 0:
                    transcript_dict[first]["identity"] = float(max_len_ident[1])
                    transcript_dict[first]["align_length"] = float(max_len_ident[2])
                    transcript_dict[first]["align_perc_len"] = float(transcript_dict[first]["align_length"]/transcript_dict[first]["length"])
                    transcript_dict[first]["align_perc_orf"] = (0 if transcript_dict[first]["orf_length"] == 0 else float(transcript_dict[first]["align_length"]/transcript_dict[first]["orf_length"]))
                score = [float(row[9])]
                first = row[0].upper()
                with_len = [[first, float(row[1]), float(row[2]), float(row[3]), float(row[9])]]
        transcript_dict[first]["identity"] = float(0)
        transcript_dict[first]["align_length"] = float(0)
        max_value = max(score)
        max_index = score.index(max_value)
        max_len_ident = with_len[max_index]
        if max_len_ident[3] > 0:
            transcript_dict[first]["identity"] = float(max_len_ident[1])
            transcript_dict[first]["align_length"] = float(max_len_ident[2])
    #fin de código adaptado de https://github.com/gbgolding/crema/blob/master/bin/featuresetup_module.py
    
    dump(transcript_dict, archivo_salida)

def generar_features(archivo_entrada, features_base, archivo_cpat, archivo_salida):
    transcritos = util_fasta.leer_fasta(archivo_entrada)
    features_globales = load(features_base)
    transcript_dict = {}
    for k in transcritos.keys():
        transcript_dict[k.strip().upper()] = {
            "length" : features_globales[k.strip().upper()]["length"],
            "gc" : features_globales[k.strip().upper()]["gc"],
            "orf_length" : features_globales[k.strip().upper()]["orf_length"],
            "orf_coverage" : features_globales[k.strip().upper()]["orf_coverage"],
            "hexamer_score" : float(0),
            "fickett_score" : float(0),
            "identity" : features_globales[k.strip().upper()]["identity"],
            "align_length" : features_globales[k.strip().upper()]["align_length"],
            "align_perc_len" : features_globales[k.strip().upper()]["align_perc_len"],
            "align_perc_orf" : features_globales[k.strip().upper()]["align_perc_orf"]
        }
    
    with open(archivo_cpat, "r") as f:
        cpat_reader = csv.reader(f, delimiter=("\t"))
        next(cpat_reader, None) # skip header
        for row in cpat_reader:
            cod_secuencia = row[0].strip().upper()
            if cod_secuencia in transcript_dict:
                transcript_dict[cod_secuencia]["fickett_score"] = float(row[3])
                transcript_dict[cod_secuencia]["hexamer_score"] = float(row[4])
                
    dump(transcript_dict, archivo_salida)

Overwriting ./libs/util_caracteristicas.py


In [ ]:
%%writefile ./libs/util_modelo_referencial.py
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import util_caracteristicas, util_fasta
from sklearn.preprocessing import StandardScaler
from sklearn.externals.joblib import dump, load
from sklearn.utils import shuffle
import random

import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import string
import os
import hashlib

class GeneradorFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, identificador=None, X_train=None,y_train=None):
        #generar todos los features
        self.identificador = identificador
        if X_train is None or y_train is None:
            return
        codigos_lncRNA = {}
        codigos_PCT = {}
        for i in range(len(X_train)):
            if y_train[i] == 0:
                codigos_PCT[X_train[i][0]] = X_train[i][1]
            else:
                codigos_lncRNA[X_train[i][0]] = X_train[i][1]
        util_caracteristicas.generar_modelo_CPAT(identificador, codigos_lncRNA, codigos_PCT)
        util_caracteristicas.generar_caracteristicas(identificador, {**codigos_lncRNA, **codigos_PCT})
        
    def fit(self, X, y=None):
        #ejecutar cpat para el conjunto de datos
        identificador = self.identificador
        self._id_cpat = identificador + "_fold_" + hashlib.sha224(''.join([x[0] for x in X]).encode()).hexdigest()#str(hash(frozenset({x[0]:1 for x in X})))
        id_cpat = self._id_cpat
        if util_caracteristicas.existe_modelo_cpat(id_cpat):
            return self
        codigos_lncRNA = {}
        codigos_PCT = {}
        for i in range(len(y)):
            if y[i] == 0:
                codigos_PCT[X[i][0]] = X[i][1]
            else:
                codigos_lncRNA[X[i][0]] = X[i][1]
        util_caracteristicas.generar_modelo_CPAT(id_cpat, codigos_lncRNA, codigos_PCT)
        util_caracteristicas.generar_caracteristicas_cpat(identificador, {**codigos_lncRNA, **codigos_PCT})
        return self
        
    def transform(self, X):
        #obtener caracteristicas
        identificador = self.identificador
        id_cpat = self._id_cpat
        codigos = {codigo[0]:codigo[1] for codigo in X}
        util_caracteristicas.generar_caracteristicas_cpat(id_cpat, codigos)
        dict_features = util_caracteristicas.obtener_caracteristicas(identificador, id_cpat, codigos)

        return [list(x.values()) for x in dict_features.values()]

def crear_modelo_referencial(identificador, tuned_parameters, scores, n_jobs, cv):
    if not os.path.isdir("./modelos_referenciales"):
        os.mkdir("./modelos_referenciales")
    
    codigos_lncRNA = util_fasta.leer_fasta("./data/" + identificador + ".lncRNA.fasta")
    codigos_PCT = util_fasta.leer_fasta("./data/" + identificador + ".PCT.fasta")
    
    X = list(codigos_lncRNA.items()) + list(codigos_PCT.items())
    y = ([1] * len(codigos_lncRNA)) + ([0] * len(codigos_PCT))
    X_train, y_train = X, y #shuffle(X, y, random_state=0)
    svm_pipeline = Pipeline(steps=[('features', GeneradorFeatures(identificador, X_train, y_train)), ('svc', SVC())])
    
    for score in scores:
        clf = GridSearchCV(svm_pipeline, tuned_parameters, cv=cv, scoring=score, n_jobs=n_jobs, refit="accuracy")
        clf.fit(X_train, y_train)
        resultado = {
            "accuracy" : clf.cv_results_['mean_test_accuracy'][clf.best_index_],
            "precision" : clf.cv_results_['mean_test_precision'][clf.best_index_],
            "recall" : clf.cv_results_['mean_test_recall'][clf.best_index_]
        }
        dump(resultado, './modelos_referenciales/resultado_{}.bin'.format(identificador))
        dump(clf.best_estimator_, './modelos_referenciales/modelo_{}.plk'.format(identificador), compress = 1)
        
        #means = clf.cv_results_['mean_test_accuracy']
        #stds = clf.cv_results_['std_test_accuracy']
        #for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        #    print("%0.3f (+/-%0.03f) for %r"
        #          % (mean, std * 2, params))
        #print()

        #print("Detailed classification report:")
        #print()
        #print("The model is trained on the full development set.")
        #print("The scores are computed on the full evaluation set.")
        #print()
        #y_true, y_pred = y_test, clf.predict(X_test)
        #print(classification_report(y_true, y_pred))
        #print()

In [ ]:
%%writefile ./libs/util_modelo_final.py